In [4]:
import re
from bs4 import BeautifulSoup
import requests
import csv
import time

In [5]:
headers = {'User-Agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
}

# Get page urls for search result

In [6]:
def get_url(job_title,city,state_abbrev):
    
    """
    
    To get the each page urls
    
    """
       
    # Set dictionary for urls
    # It varies depends what job title, location you input
    # For the project I just pass the search conditions in arguments
    
    
    # Dictionary keys
    keys = {'search_title':[job_title],'city':[city],'state':[state_abbrev]}
    
    # Actual working url
    url = f"https://www.indeed.com/jobs?q={keys['search_title'][0]}&l={keys['city'][0]}%2C%20{keys['state'][0]}&radius=0&sort=date&start=0&vjk"
    
    # url for web scraping, city to save in records in get_info function
    return url, city

# Get info from the result pages 
    title, company name, rate, location, remote, url

In [7]:
def get_info(url, city):
    
    """
    
    To gather data of interest; Job title, Company name, Rate, Location, Remote
    
    """
    
    # Get text from input url
    response = requests.get(url, headers = headers)
    text = BeautifulSoup(response.text, 'html.parser')
        
    # From the script, find "resultContent"
    # This will give you the same amount of the search results
    result_contents = text.find_all('td', 'resultContent')
    
    # content_records to save the results for all pages
    # comp_webpage is each company's website in indeed.com to get detail rate/satisfaction information
    content_records = []
    comp_webpage = []
    
    
    
    
    # Usually each page has 15 results of job searching
    # len(result_contents) should be 15 in normal
    # get title, company name, rate, location, remote for each result
    
    for x in range(len(result_contents)):
        result_contentN = result_contents[x]
        comp_url_N = ''

        title_for_N = result_contentN.select(".resultContent span[title]")[0].string

        name_for_N = result_contentN.find('span','companyName').string

        rate_result = result_contentN.find('span', attrs = {'aria-hidden':'true'})
        rate_N = re.sub("<.*?>", "", str(rate_result))

        location_for_N = city

        remote_result = result_contentN.find('span', attrs = {'class':'remote-bullet'})
        remote_N = re.sub("<.*?>", "", str(remote_result))

                        
        
        # company's website url in indeed.com (if any) and store it
        for link in result_contentN.find('span','companyName').find_all('a', href=True):
            comp_url_N = 'https://www.indeed.com' + link['href']            
            comp_webpage.append(comp_url_N)
        
        
        
        # store the data
        info_record = [title_for_N,name_for_N,rate_N,location_for_N,remote_N,comp_url_N]
        content_records.append(info_record)
    
    
    # to the next search result page
    next_url = ''
    
    try:
        next_url = text.find('a',{'aria-label':'Next'}).get('href')
    except AttributeError:
        next_url = 'last page'
    
    
    # to see where the code is working
    print(next_url)
    
    # To avoid accessing the website too often in a short time
    # Still it leads us recaptcha at some point
    time.sleep(9)
    
    # return the stored data, the next page url, the companys' website url
    return content_records, next_url, comp_webpage

# Do web scraping for all the search result pages

In [8]:
# For this project, I input the search conditions manually
def project1(title, city, state):
    page_url, city = get_url(title,city,state)
    
    
    # to store all data from the results
    total_records = []
    web_records = []
    
    
    
    # Do web scraping and store data until there's no more next page
    while True:    
        temp_record, partial_url, comp_webpage = get_info(page_url, city)
        total_records.append(temp_record)
        
        if partial_url == 'last page':
            break
        else:
            page_url = 'https://www.indeed.com' + partial_url
            
        web_records.append(comp_webpage)
        
        
        # To avoid accessing the website too often in a short time
        # Still it leads us recaptcha at some point
        time.sleep(19)
            
    
    
    
    # Save data into a csv file
    with open(title + " " + city +'.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerow(['JobTitle','Company','Rate','Location','Remote','url address'])
        for info in total_records:
            writer.writerows(info)
    
    
    # return company's website urls
    return web_records

# Actual running

In [9]:
# 2 job titles, 6 different cities
job_title = ('data scientist','web developer')
city_list = ("New York","Los Angeles","Chicago","Houston","Seattle","Phoenix")
state_list = ('NY','CA','IL','TX','WA','AZ')

webpage = []

# get all url addresses for each serach input
for title in job_title:
    for it in range(len(city_list)):
        webpage.append(project1(title,city_list[it],state_list[it]))

/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=10
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=20
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=30
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=40
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=50
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=60
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=70
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=80
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=90
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=100
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=110
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=120
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=130
/jobs?q=data+scientist&l=New+York%2C+NY&radius=0&sort=date&start=140
/jobs?q=data+scientist&l=New+York%2C+NY&rad

/jobs?q=data+scientist&l=Seattle%2C+WA&radius=0&sort=date&start=230
/jobs?q=data+scientist&l=Seattle%2C+WA&radius=0&sort=date&start=240
last page
/jobs?q=data+scientist&l=Phoenix%2C+AZ&radius=0&sort=date&start=10
/jobs?q=data+scientist&l=Phoenix%2C+AZ&radius=0&sort=date&start=20
/jobs?q=data+scientist&l=Phoenix%2C+AZ&radius=0&sort=date&start=30
/jobs?q=data+scientist&l=Phoenix%2C+AZ&radius=0&sort=date&start=40
/jobs?q=data+scientist&l=Phoenix%2C+AZ&radius=0&sort=date&start=50
/jobs?q=data+scientist&l=Phoenix%2C+AZ&radius=0&sort=date&start=60
last page
/jobs?q=web+developer&l=New+York%2C+NY&radius=0&sort=date&start=10
/jobs?q=web+developer&l=New+York%2C+NY&radius=0&sort=date&start=20
/jobs?q=web+developer&l=New+York%2C+NY&radius=0&sort=date&start=30
/jobs?q=web+developer&l=New+York%2C+NY&radius=0&sort=date&start=40
/jobs?q=web+developer&l=New+York%2C+NY&radius=0&sort=date&start=50
/jobs?q=web+developer&l=New+York%2C+NY&radius=0&sort=date&start=60
/jobs?q=web+developer&l=New+York%2C+NY&r

/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=290
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=300
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=310
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=320
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=330
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=340
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=350
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=360
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=370
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=380
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=390
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=400
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=410
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start=420
/jobs?q=web+developer&l=Chicago%2C+IL&radius=0&sort=date&start

# Do Web Scraping in company websites

In [11]:
def company_info(webpage):
            
    # list - flattening for company websites in indeed.com
    list_webpage = [item for link in webpage for item in link]
    total_comp_link = [item for link in list_webpage for item in link]
    
    
    # remove the duplicated company urls
    fin_comp_list = []
    for i in total_comp_link:
        if i not in fin_comp_list:
            fin_comp_list.append(i)
        
        
    # To save data from company websites into a csv file
    # Make a csv file first
    with open('rate and salary.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerow(['Name','Overall','Rate 5','Rate 4','Rate 3','Rate 2','Rate 1','Satisfaction','url'])
    
    
    
    # get review and salary info
    for x in range(len(fin_comp_list)):
         
        # get company name
        comp_name = fin_comp_list[x][27:]
 
        # store company url
        comp_url = fin_comp_list[x]
        
    
    

        # reviews
        
        # url for review
        r_url = fin_comp_list[x] + '/reviews' 
        
        # get text
        response_r = requests.get(r_url, headers = headers)
        text_r = BeautifulSoup(response_r.text, 'html.parser') 
        
        # get overall review point
        comp_rate_overall = text_r.find('h2','css-18uru8m e1tiznh50').string
        
        # get detail review point (count of 1-5 stars)
        star_num = []
        comp_rate_detail = text_r.find('div','css-vktqis eu4oa1w0').find_all('span','css-xvmbeo e1wnkr790')[-5:]
        for i in comp_rate_detail:
            star_num.append(i.string)

            
            
        # salaries
        
        # url for salary
        s_url = fin_comp_list[x] + '/salaries'
        
        # get text
        response_s = requests.get(s_url, headers = headers)
        text_s = BeautifulSoup(response_s.text, 'html.parser')
        
        # get overall salary satisfaction percentage
        if text_s.find('div',{'class':'cmp-SalarySatisfactionSidebarWidgetPieChart'}) is not None:
            comp_percentage_overall = text_s.find('div','cmp-SalarySatisfactionSidebarWidgetPieChart-inside').string
        else:
            comp_percentage_overall = "NA"
        
        # store information
        company_info = [[comp_name,comp_rate_overall,
                         star_num[0],star_num[1],star_num[2],star_num[3],star_num[4],
                         comp_percentage_overall,comp_url]]
        
        
        # to track the progress
        print(x, company_info)
        
        
        # use 'a' mode to append the result into the created csv file
        with open('rate and salary1.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f, delimiter=',')
            writer.writerows(company_info)
        
        # To avoid accessing the website too often in a short time
        time.sleep(45)
        
                

# Actual running #2

In [12]:
company_info(webpage)

0 [['Square', '3.7', '46', '29', '12', '9', '17', '64%', 'https://www.indeed.com/cmp/Square']]
1 [['Accenture', '4.0', '7.3K', '8.5K', '3.6K', '857', '703', '78%', 'https://www.indeed.com/cmp/Accenture']]
2 [['United-Nations', '4.4', '497', '213', '77', '16', '14', '73%', 'https://www.indeed.com/cmp/United-Nations']]
3 [['Chase-2', '3.9', '3K', '2.8K', '1.9K', '506', '449', '56%', 'https://www.indeed.com/cmp/Chase-2']]
4 [['Zscaler', '3.9', '9', '9', '4', '2', '1', 'NA', 'https://www.indeed.com/cmp/Zscaler']]
5 [['Sirius-Xm-Radio', '3.5', '80', '75', '64', '24', '32', '47%', 'https://www.indeed.com/cmp/Sirius-Xm-Radio']]
6 [['Human-Rights-Watch', '4.8', '21', '3', '1', '0', '0', '55%', 'https://www.indeed.com/cmp/Human-Rights-Watch']]
7 [['Verizon', '3.9', '10.3K', '9.8K', '6K', '1.8K', '1.4K', '71%', 'https://www.indeed.com/cmp/Verizon']]
8 [['Amazon.com', '3.5', '22.4K', '20K', '17.4K', '7.9K', '9.4K', '52%', 'https://www.indeed.com/cmp/Amazon.com']]
9 [['Nielsen', '3.8', '973', '905

74 [['Nyu-Langone-Health', '4.0', '913', '694', '318', '105', '105', '69%', 'https://www.indeed.com/cmp/Nyu-Langone-Health']]
75 [['Ogilvy', '4.1', '299', '269', '124', '28', '22', '60%', 'https://www.indeed.com/cmp/Ogilvy']]
76 [['Ey', '4.0', '2.7K', '3.2K', '1.6K', '335', '238', '61%', 'https://www.indeed.com/cmp/Ey']]
77 [['American-Civil-Liberties-Union', '4.3', '63', '18', '11', '6', '3', '74%', 'https://www.indeed.com/cmp/American-Civil-Liberties-Union']]
78 [['Harnham', '4.6', '6', '2', '1', '0', '0', 'NA', 'https://www.indeed.com/cmp/Harnham']]
79 [['Alliancebernstein', '3.5', '60', '63', '57', '26', '20', '45%', 'https://www.indeed.com/cmp/Alliancebernstein']]
80 [['Motivate-LLC', '3.6', '3', '1', '3', '2', '0', '56%', 'https://www.indeed.com/cmp/Motivate-LLC']]
81 [['J.-Crew-Group,-Inc.', '3.8', '385', '392', '244', '77', '93', '58%', 'https://www.indeed.com/cmp/J.-Crew-Group,-Inc.']]
82 [['Weill-Cornell-Medicine-aed1176c', '4.0', '290', '207', '111', '40', '25', '57%', 'http

149 [['Guidewire-Software,-Inc.', '3.7', '15', '28', '8', '6', '4', '47%', 'https://www.indeed.com/cmp/Guidewire-Software,-Inc.']]
150 [['Databricks', '3.3', '1', '1', '0', '0', '1', 'NA', 'https://www.indeed.com/cmp/Databricks']]
151 [['Abbvie', '4.0', '468', '346', '203', '73', '54', '70%', 'https://www.indeed.com/cmp/Abbvie']]
152 [['Apple', '4.2', '5.2K', '3.1K', '1.5K', '482', '357', '68%', 'https://www.indeed.com/cmp/Apple']]
153 [['Rauxa', '3.4', '4', '5', '5', '3', '1', '80%', 'https://www.indeed.com/cmp/Rauxa']]
154 [['Brooks-Brothers', '3.7', '205', '202', '152', '35', '48', '47%', 'https://www.indeed.com/cmp/Brooks-Brothers']]
155 [['Hospital-For-Special-Surgery', '4.2', '231', '125', '47', '17', '19', '77%', 'https://www.indeed.com/cmp/Hospital-For-Special-Surgery']]
156 [['Ripple', '3.5', '3', '2', '3', '3', '0', '75%', 'https://www.indeed.com/cmp/Ripple']]
157 [['S&P-Global', '3.9', '475', '495', '274', '70', '50', '71%', 'https://www.indeed.com/cmp/S&P-Global']]
158 [['W

224 [['Medidata-Solutions', '3.6', '20', '33', '22', '5', '7', '67%', 'https://www.indeed.com/cmp/Medidata-Solutions']]
225 [['Bees', '4.0', '3', '1', '1', '1', '0', 'NA', 'https://www.indeed.com/cmp/Bees']]
226 [['Anheuser--busch-Inbev', '3.7', '525', '469', '324', '148', '115', '63%', 'https://www.indeed.com/cmp/Anheuser--busch-Inbev']]
227 [['General-Assembly', '4.2', '111', '78', '30', '5', '7', '66%', 'https://www.indeed.com/cmp/General-Assembly']]
228 [['Netcom-Learning', '4.1', '28', '4', '2', '1', '7', '20%', 'https://www.indeed.com/cmp/Netcom-Learning']]
229 [['D.-E.-Shaw-&-Co.---Investment-Firm', '4.1', '23', '24', '11', '2', '0', '78%', 'https://www.indeed.com/cmp/D.-E.-Shaw-&-Co.---Investment-Firm']]
230 [['Bounce-Exchange', '4.5', '5', '0', '0', '1', '0', 'NA', 'https://www.indeed.com/cmp/Bounce-Exchange']]
231 [['Nyc-Data-Science-Academy', '4.6', '12', '5', '1', '0', '0', 'NA', 'https://www.indeed.com/cmp/Nyc-Data-Science-Academy']]
232 [['Source-Enterprises', '4.3', '2',

299 [['The-University-of-Chicago', '4.1', '287', '210', '102', '33', '25', '62%', 'https://www.indeed.com/cmp/The-University-of-Chicago']]
300 [['United-Airlines', '4.0', '1.6K', '1.2K', '684', '205', '200', '63%', 'https://www.indeed.com/cmp/United-Airlines']]
301 [['American-College-of-Surgeons', '3.2', '5', '6', '2', '6', '3', '66%', 'https://www.indeed.com/cmp/American-College-of-Surgeons']]
302 [['Iri', '3.6', '20', '24', '20', '6', '7', '40%', 'https://www.indeed.com/cmp/Iri']]
303 [['Cardinal-Intellectual-Property', '4.2', '21', '4', '3', '4', '1', '30%', 'https://www.indeed.com/cmp/Cardinal-Intellectual-Property']]
304 [['Bissell', '3.5', '21', '24', '12', '9', '11', '52%', 'https://www.indeed.com/cmp/Bissell']]
305 [['Riskonnect', '3.3', '3', '4', '7', '1', '2', '52%', 'https://www.indeed.com/cmp/Riskonnect']]
306 [['Glassdoor', '4.2', '23', '10', '3', '3', '2', '68%', 'https://www.indeed.com/cmp/Glassdoor']]
307 [['Numerator-1', '3.6', '33', '31', '38', '12', '7', '47%', 'htt

...

740 [['Absolute-Web', '5.0', '7', '0', '0', '0', '0', 'NA', 'https://www.indeed.com/cmp/Absolute-Web']]
741 [['The-Sulfur-Group', '3.0', '1', '0', '0', '0', '1', 'NA', 'https://www.indeed.com/cmp/The-Sulfur-Group']]
742 [['Ciplex', '3.7', '1', '0', '2', '0', '0', 'NA', 'https://www.indeed.com/cmp/Ciplex']]
743 [['Ad-Hoc-Team', '5.0', '2', '0', '0', '0', '0', 'NA', 'https://www.indeed.com/cmp/Ad-Hoc-Team']]
744 [['Social-Tables', '3.8', '1', '1', '2', '0', '0', 'NA', 'https://www.indeed.com/cmp/Social-Tables']]
745 [['Antra,-Inc', '3.0', '1', '0', '2', '0', '1', 'NA', 'https://www.indeed.com/cmp/Antra,-Inc']]
746 [['Awin', '3.3', '2', '1', '2', '3', '0', 'NA', 'https://www.indeed.com/cmp/Awin']]
747 [['Depaul-University', '4.3', '216', '109', '41', '15', '9', '74%', 'https://www.indeed.com/cmp/Depaul-University']]
748 [['Radancy', '3.7', '43', '48', '32', '16', '8', '41%', 'https://www.indeed.com/cmp/Radancy']]
749 [['Comcast', '3.7', '5.3K', '5.1K', '3.9K', '1.4K', '1.4K', '54%', 'http

816 [['Cdm-Smith', '3.8', '90', '75', '44', '24', '16', '65%', 'https://www.indeed.com/cmp/Cdm-Smith']]
817 [['Wipro', '3.8', '4.1K', '6K', '3.6K', '817', '596', '41%', 'https://www.indeed.com/cmp/Wipro']]
818 [['Franklin-Energy', '3.5', '15', '14', '15', '5', '5', '57%', 'https://www.indeed.com/cmp/Franklin-Energy']]
819 [['Jacobs', '3.9', '1.8K', '1.8K', '894', '286', '257', '72%', 'https://www.indeed.com/cmp/Jacobs']]
820 [['Kizan-Technologies', '4.0', '2', '3', '0', '1', '0', '80%', 'https://www.indeed.com/cmp/Kizan-Technologies']]
821 [['BGRS', '3.4', '28', '30', '30', '10', '14', '49%', 'https://www.indeed.com/cmp/BGRS']]
822 [['Tricentis', '3.1', '2', '1', '3', '0', '2', 'NA', 'https://www.indeed.com/cmp/Tricentis']]
823 [['Mediaocean', '3.5', '8', '9', '11', '3', '2', '71%', 'https://www.indeed.com/cmp/Mediaocean']]
824 [['The-Art-Institute-of-Chicago', '3.8', '30', '21', '19', '10', '4', '34%', 'https://www.indeed.com/cmp/The-Art-Institute-of-Chicago']]
825 [['Gelber-Group', '

890 [['Lone-Star-College', '4.3', '366', '198', '92', '17', '11', '68%', 'https://www.indeed.com/cmp/Lone-Star-College']]
891 [['Mandiant', '3.5', '1', '1', '1', '1', '0', 'NA', 'https://www.indeed.com/cmp/Mandiant']]
892 [['Detail-Management', '4.0', '6', '0', '1', '1', '1', 'NA', 'https://www.indeed.com/cmp/Detail-Management']]
893 [['The-Art-Institute-of-Houston', '3.3', '6', '12', '11', '9', '1', '63%', 'https://www.indeed.com/cmp/The-Art-Institute-of-Houston']]
894 [['Foley-Carrier-Services-LLC', '3.1', '17', '17', '13', '15', '14', '43%', 'https://www.indeed.com/cmp/Foley-Carrier-Services-LLC']]
895 [['Waste-Management-1', '3.4', '1.4K', '1.4K', '1.2K', '606', '775', '48%', 'https://www.indeed.com/cmp/Waste-Management-1']]
896 [['Alacrity-Solutions', '3.2', '5', '1', '2', '1', '4', '81%', 'https://www.indeed.com/cmp/Alacrity-Solutions']]
897 [['Goco', '2.2', '1', '1', '1', '2', '4', '45%', 'https://www.indeed.com/cmp/Goco']]
898 [['Twin-Eagle', '3.5', '2', '5', '2', '1', '1', '66

962 [['Belcan', '3.6', '171', '211', '156', '62', '57', '57%', 'https://www.indeed.com/cmp/Belcan']]
963 [['City-of-Seattle', '3.8', '73', '51', '39', '15', '14', '79%', 'https://www.indeed.com/cmp/City-of-Seattle']]
964 [['Kaiser-Permanente', '4.1', '5.8K', '3.2K', '1.7K', '592', '567', '82%', 'https://www.indeed.com/cmp/Kaiser-Permanente']]
965 [['Itrellis', '5.0', '3', '0', '0', '0', '0', 'NA', 'https://www.indeed.com/cmp/Itrellis']]
966 [['Avalara', '2.9', '13', '13', '8', '16', '16', '54%', 'https://www.indeed.com/cmp/Avalara']]
967 [['Tideworks-Technology', '3.7', '7', '2', '2', '2', '2', '92%', 'https://www.indeed.com/cmp/Tideworks-Technology']]
968 [['Zenoti', '4.6', '8', '1', '0', '1', '0', 'NA', 'https://www.indeed.com/cmp/Zenoti']]
969 [['Catalysis', '4.0', '2', '2', '2', '0', '0', 'NA', 'https://www.indeed.com/cmp/Catalysis']]
970 [['Dorsey-&-Whitney', '3.8', '23', '14', '13', '4', '4', '62%', 'https://www.indeed.com/cmp/Dorsey-&-Whitney']]
971 [['Office-Depot', '3.5', '3.3

1035 [['Desert-Financial-Credit-Union', '3.4', '49', '34', '37', '21', '22', '49%', 'https://www.indeed.com/cmp/Desert-Financial-Credit-Union']]
1036 [['Equality-Health', '3.1', '3', '2', '1', '1', '3', '60%', 'https://www.indeed.com/cmp/Equality-Health']]
1037 [['Bamboohr', '4.4', '41', '5', '5', '0', '5', '53%', 'https://www.indeed.com/cmp/Bamboohr']]
1038 [['Lexmark', '3.9', '162', '166', '103', '37', '16', '76%', 'https://www.indeed.com/cmp/Lexmark']]
1039 [['Corvel-Corporation', '3.0', '128', '131', '138', '88', '149', '45%', 'https://www.indeed.com/cmp/Corvel-Corporation']]
1040 [['Snell-&-Wilmer', '2.7', '4', '4', '2', '3', '8', '53%', 'https://www.indeed.com/cmp/Snell-&-Wilmer']]
1041 [['Shopify', '3.9', '48', '26', '11', '11', '10', '73%', 'https://www.indeed.com/cmp/Shopify']]
1042 [['Arizona-Supreme-Court', '3.6', '6', '12', '6', '2', '2', '53%', 'https://www.indeed.com/cmp/Arizona-Supreme-Court']]
1043 [['Benefitfocus.com', '3.0', '30', '40', '58', '37', '27', '45%', 'https